**Image preparation**

Firstly all libaries will be imported.

In [5]:
import cv2 as cv
import os
import numpy as np

Define input and output folders for the processed and cropped images so the images can be prepared for the model

In [6]:
input_folders = ["Dataset/Plaster", "Dataset/Machine", "Dataset/Philips", "Dataset/Torx"]
output_folders = ["Processed/Plaster", "Processed/Machine", "Processed/Philips", "Processed/Torx"]
cropped_folders = ["Cropped/Plaster", "Cropped/Machine", "Cropped/Philips", "Cropped/Torx"]

# Create output folders if they don't exist
for folder in output_folders + cropped_folders:
    os.makedirs(folder, exist_ok=True)

When all the output folders are created, the kernel used for blob dilation will be defined. Here it is important to use a odd-number for dilation size.

In [7]:
# Define a kernel for dilation
kernel = np.ones((9,9), np.uint8)  # Adjust size if needed

With the dilation kernel defined, each image will be processed using grayscale, dilation, contours and will be cropped from the original image.

In [8]:
for input_folder, output_folder, cropped_folder in zip(input_folders, output_folders, cropped_folders):
    for filename in os.listdir(input_folder):
        if filename.endswith((".jpg")):

            # Read the image from the dataset
            img_path = os.path.join(input_folder, filename)
            img = cv.imread(img_path)

            # Convert the image to grayscale
            gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

            # Apply a threshold to create a blob on the subject
            _, thresh = cv.threshold(gray, 100, 255, cv.THRESH_BINARY_INV)

            # The dilation will be applied to ensure all parts of the screw remains
            dilated = cv.dilate(thresh, kernel, iterations=2)

            # The processed image is saved
            output_path = os.path.join(output_folder, filename)
            cv.imwrite(output_path, dilated)

                        # Using contours, the blobs are detected
            contours, _ = cv.findContours(dilated, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

            # Desired crop size
            crop_size = 256
            half_crop = crop_size // 2

            for i, contour in enumerate(contours):
                x, y, w, h = cv.boundingRect(contour)

                # Calculate the midpoint of the blob
                center_x = x + w // 2
                center_y = y + h // 2

                # Define top-left corner of the 256x256 crop centered at the midpoint
                x_start = max(center_x - half_crop, 0)
                y_start = max(center_y - half_crop, 0)

                # Ensure we don't exceed image boundaries
                x_end = min(x_start + crop_size, img.shape[1])
                y_end = min(y_start + crop_size, img.shape[0])

                # Adjust start if the crop goes out of bounds (important at image edges)
                x_start = max(x_end - crop_size, 0)
                y_start = max(y_end - crop_size, 0)

                # Crop the fixed-size image
                cropped = img[y_start:y_end, x_start:x_end]

                # Save the cropped image
                cropped_filename = f"{filename.split('.')[0]}_crop_{i}.png"
                cropped_path = os.path.join(cropped_folder, cropped_filename)
                cv.imwrite(cropped_path, cropped)